# Quantitative Value Strategy for S&P500

Building an investing strategy that selects the 50 stocks with the best value metrics and recommends trades for an equal-weight portfolio of these 50 stocks.

## Basic Strategy

### Library Imports

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
from scipy import stats
from scipy.stats import percentileofscore as score
from statistics import mean
import requests
from secrets import IEX_CLOUD_API_TOKEN

### Importing Our List of Stocks

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


### Checking returned data from different API Calls


In [3]:
symbol = "AAPL"
url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url).json()
data

{'avgTotalVolume': 71216788,
 'calculationPrice': 'close',
 'change': 0.05,
 'changePercent': 0.0003,
 'close': 168.12,
 'closeSource': 'lcofiafi',
 'closeTime': 1707461082373,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 166.42,
 'delayedPriceTime': 1677185036785,
 'extendedChange': 0.09,
 'extendedChangePercent': 0.00056,
 'extendedPrice': 165.27,
 'extendedPriceTime': 1701339313139,
 'high': 171,
 'highSource': 'meeitpe l1iy5da  ecnrdu',
 'highTime': 1682450114501,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 168.27,
 'iexCloseTime': 1735769851438,
 'iexLastUpdated': 1704438056396,
 'iexMarketPercent': 0.02123003655656669,
 'iexOpen': 164.99,
 'iexOpenTime': 1687999191043,
 'iexRealtimePrice': 168.78,
 'iexRealtimeSize': 25,
 'iexVolume': 1290522,
 'lastTradeTime': 1734849695533,
 'latestPrice': 169.86,
 'latestSource': 'Close',
 'latestTime': 'August 9, 2022',
 'latestUpdate': 1715261139634,
 'latestVolume': 63674632,
 

In [4]:
data["peRatio"]

27.93

### Executing Batch API Calls & Populating Our DataFrame

In [5]:
def breakdown(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(breakdown(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(i, ":", symbol_strings[i], "\n")

cols = ['Ticker', 'Stock Price', 'P/E Ratio', 'No. of Shares to Buy']

0 : A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA 

1 : CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS 

2 : GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,I

In [6]:
updated_stocks = pd.DataFrame(columns=cols)

for string in symbol_strings:
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    for stock in string.split(","):
        updated_stocks = updated_stocks.append(
        pd.Series([
            stock,
            data[stock]["quote"]['latestPrice'],
            data[stock]["quote"]["peRatio"],
            "N/A"
        ], index=cols), ignore_index=True)
        
updated_stocks

,Ticker,Stock Price,P/E Ratio,No. of Shares to Buy
0,A,132.06,31.61,N/A
1,AAL,14.84,-5.21,N/A
2,AAP,193.24,21.45,N/A
3,AAPL,171.54,27.51,N/A
4,ABBV,146.63,20.65,N/A
...,...,...,...,...
496,YUM,122.15,21.34,N/A
497,ZBH,115.80,105.89,N/A
498,ZBRA,333.08,34.57,N/A
499,ZION,56.61,6.36,N/A


### Removing Glamour Stocks

Note that the lower the P/E Ratio, the better, as it points towards stocks that have rather high earnings per share. Therefore, here, we'll keep the 50 lowest **positive** P/E Ratios and remove all others

In [7]:
updated_stocks = updated_stocks[updated_stocks["P/E Ratio"] > 0] # removing negative P/E Ratios
updated_stocks.sort_values(by="P/E Ratio", inplace=True)
updated_stocks = updated_stocks[:50]
updated_stocks.reset_index(drop=True, inplace=True)
updated_stocks

c:\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Ticker,Stock Price,P/E Ratio,No. of Shares to Buy
0,NRG,39.58,2.8,N/A
1,APA,33.13,3.8,N/A
2,KSS,30.27,4.53,N/A
3,PVH,64.92,4.61,N/A
4,COF,111.13,4.65,N/A
5,AIG,55.83,4.7,N/A
6,PHM,42.46,4.75,N/A
7,SYF,34.60,4.95,N/A
8,DHI,77.21,4.96,N/A
9,MGM,34.85,4.98,N/A


### Calculating the Number of Shares to Buy

In [8]:
def calculate_shares(df):
    portfolio_size = input("Enter the value of your portfolio (as a number):")
    check = False
    
    while check is False:
        try:
            val = float(portfolio_size)
            check = True
        except ValueError:
            print("That is not a number! Please try again:")
            portfolio_size = input("Enter the value of your portfolio (as a number):")
            
    val = float(portfolio_size)
    
    position_size = val / len(df.index)
    
    print(f"\nGiven your portfolio value at ${val}, you're position size is {position_size}\n")
    
    for i in range(len(df.index)):
        price = df.loc[i, "Stock Price"]
        df.loc[i, "No. of Shares to Buy"] = position_size // price

In [9]:
calculate_shares(updated_stocks)


Given your portfolio value at $123456.0, you're position size is 2469.12



In [10]:
updated_stocks

,Ticker,Stock Price,P/E Ratio,No. of Shares to Buy
0,NRG,39.58,2.8,62.0
1,APA,33.13,3.8,74.0
2,KSS,30.27,4.53,81.0
3,PVH,64.92,4.61,38.0
4,COF,111.13,4.65,22.0
5,AIG,55.83,4.7,44.0
6,PHM,42.46,4.75,58.0
7,SYF,34.60,4.95,71.0
8,DHI,77.21,4.96,31.0
9,MGM,34.85,4.98,70.0


## Improved Strategy

Earlier, we had just used Price-to-Earnings ratio to determine which stocks to buy but that is not a practical approach. All approaches have their flaws e.g. P/E Ratio doesn't work well with stocks with negative earnings and P/B Ratio doesn't perform well with stocks that buyback their own shares. Therefore, we need to take into account composite of values and then use percentiles to build robust quantitative value strategies. The values we'll take into account now are:

* Price-to-earnings ratio (stock price / earnings per share)
* Price-to-book ratio (stock price / shareholder's equity per share)
* Price-to-sales ratio (stock price / revenue per share)
* EV-to-EBITDA ratio
* EV-to-GP ratio

EV is Enterprise value, which is the amount one would have to pay to buy the entire company in full, including market cap., debt and minority shares etc.

EBITDA is Earnings Before Interest, Taxes, Depreciation, and Amortization, which is a company's net income before reductions such as taxes and depreciation etc.

GP is Gross Profit, which is a company's profit after taking away the Cost of Goods Sold from it.

### Testing sample case and parsing data

In [11]:
symbol="AAPL"
url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url).json()

# Price-to-earnings ratio
pe = data[symbol]["quote"]["peRatio"]
print(f"PE Ratio is {pe}\n")

# Price-to-book ratio
pb = data[symbol]["advanced-stats"]["priceToBook"]
print(f"PB Ratio is {pb}\n")

# Price-to-sales ratio
ps = data[symbol]["advanced-stats"]["priceToSales"]
print(f"PS Ratio is {ps}\n")

# Enterprise Value
ev = data[symbol]["advanced-stats"]["enterpriseValue"]
print(f"EV is {ev}\n")
      
# EBITDA
ebitda = data[symbol]["advanced-stats"]["EBITDA"]
print(f"EBITDA is {ebitda}\n")

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
evToEBITDA = ev / ebitda
print(f"EV-EBITDA Ratio is {evToEBITDA}\n")

# Gross Profit
gp = data[symbol]["advanced-stats"]["grossProfit"]
print(f"GP is {gp}\n")

# Enterprise Value divided by Gross Profit (EV/GP)      
evToGP = ev / gp
print(f"EV-GP Ratio is {evToGP}")

PE Ratio is 27.65

PB Ratio is 47.4

PS Ratio is 6.94

EV is 2761838275053

EBITDA is 130173878625

EV-EBITDA Ratio is 21.216532104795

GP is 174494126924

EV-GP Ratio is 15.827686144737147


In [12]:
rv_cols = [
    "Ticker",
    "Stock Price",
    "No. of Shares to Buy",
    "RV Score",
    "P/E Ratio",
    "P/E Percentile",
    "P/B Ratio",
    "P/B Percentile",
    "P/S Ratio",
    "P/S Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GS",
    "EV/GS Percentile",
]

rv_stocks = pd.DataFrame(columns = rv_cols)
rv_stocks

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile


In [13]:
# error-proofing
def handle_errors_when_filling_df(ev, header):
    try:
        val = ev / header
    except TypeError:
        val = np.NaN
        
    return val

In [14]:
rv_stocks = pd.DataFrame(columns = rv_cols)

for string in symbol_strings:
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    for stock in string.split(","):
        ev = data[stock]["advanced-stats"]["enterpriseValue"]
        ebitda = data[stock]["advanced-stats"]["EBITDA"]
        gp = data[stock]["advanced-stats"]["grossProfit"]
        
        rv_stocks = rv_stocks.append(
            pd.Series(
                [
                    stock,
                    data[stock]["quote"]['latestPrice'],
                    "N/A",
                    "N/A",
                    data[stock]["quote"]["peRatio"],
                    "N/A",
                    data[stock]["advanced-stats"]["priceToBook"],
                    "N/A",
                    data[stock]["advanced-stats"]["priceToSales"],
                    "N/A",
                    handle_errors_when_filling_df(ev, ebitda),
                    "N/A",
                    handle_errors_when_filling_df(ev, gp),
                    "N/A",
                ], index=rv_cols), ignore_index = True)
        
rv_stocks

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile
0,A,131.67,N/A,N/A,31.96,N/A,7.75,N/A,6.08,N/A,23.205815,N/A,12.040917,N/A
1,AAL,14.90,N/A,N/A,-5.2,N/A,-1.19,N/A,0.2451,N/A,108.710243,N/A,0.846311,N/A
2,AAP,197.68,N/A,N/A,21.72,N/A,4.12,N/A,1.11,N/A,12.651153,N/A,2.610145,N/A
3,AAPL,169.50,N/A,N/A,28.44,N/A,46.6,N/A,6.95,N/A,20.905739,N/A,15.830150,N/A
4,ABBV,144.78,N/A,N/A,20.02,N/A,17.29,N/A,4.36,N/A,11.398753,N/A,7.791025,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.36,N/A,N/A,22.27,N/A,-4.16,N/A,5.12,N/A,19.914102,N/A,9.217570,N/A
497,ZBH,117.87,N/A,N/A,107.98,N/A,1.96,N/A,3.34,N/A,18.322700,N/A,5.920571,N/A
498,ZBRA,335.45,N/A,N/A,34.66,N/A,7.15,N/A,3.06,N/A,17.828416,N/A,7.323461,N/A
499,ZION,57.29,N/A,N/A,6.38,N/A,1.58,N/A,2.36,N/A,4.982772,N/A,2.368859,N/A


### Dealing With Missing Data in Our DataFrame

In [15]:
rv_stocks[rv_stocks.isnull().any(axis=1)]

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile
28,ALXN,185.300,N/A,N/A,60.63,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
40,AON,298.980,N/A,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
71,BRK.B,298.240,N/A,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
88,CERN,95.600,N/A,N/A,50.02,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
118,CTL,11.000,N/A,N/A,10.01,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
135,DISCK,25.350,N/A,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
164,ETFC,50.380,N/A,N/A,14.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
185,FLIR,59.020,N/A,N/A,33.25,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
189,FOX,32.620,N/A,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A
192,FRT,111.410,N/A,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A


In [16]:
for col in ["P/E Ratio", "P/B Ratio", "P/S Ratio", "EV/EBITDA", "EV/GS"]:
    rv_stocks[col].fillna(rv_stocks[col].mean(), inplace=True)

In [17]:
rv_stocks[rv_stocks.isnull().any(axis=1)]

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile


### Calculating Value Percentiles

In [18]:
metrics = {
    "P/E Ratio": "P/E Percentile",
    "P/B Ratio": "P/B Percentile",
    "P/S Ratio": "P/S Percentile",
    "EV/EBITDA": "EV/EBITDA Percentile",
    "EV/GS": "EV/GS Percentile"
}

for metric in metrics:
    for row in rv_stocks.index:
        rv_stocks.loc[row, metrics[metric]] = score(rv_stocks[metric], rv_stocks.loc[row, metric]) / 100
        
rv_stocks

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile
0,A,131.67,N/A,N/A,31.96,0.730539,7.75,0.777445,6.0800,0.826347,23.205815,0.822355,12.040917,0.824351
1,AAL,14.90,N/A,N/A,-5.20,0.031936,-1.19,0.061876,0.2451,0.011976,108.710243,0.996008,0.846311,0.031936
2,AAP,197.68,N/A,N/A,21.72,0.516966,4.12,0.58483,1.1100,0.179641,12.651153,0.42515,2.610145,0.165669
3,AAPL,169.50,N/A,N/A,28.44,0.648703,46.60,0.978044,6.9500,0.850299,20.905739,0.762475,15.830150,0.92016
4,ABBV,144.78,N/A,N/A,20.02,0.46507,17.29,0.932136,4.3600,0.714571,11.398753,0.357285,7.791025,0.576846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.36,N/A,N/A,22.27,0.522954,-4.16,0.057884,5.1200,0.782435,19.914102,0.740519,9.217570,0.700599
497,ZBH,117.87,N/A,N/A,107.98,0.978044,1.96,0.289421,3.3400,0.568862,18.322700,0.704591,5.920571,0.429142
498,ZBRA,335.45,N/A,N/A,34.66,0.760479,7.15,0.751497,3.0600,0.53992,17.828416,0.688623,7.323461,0.53493
499,ZION,57.29,N/A,N/A,6.38,0.093812,1.58,0.204591,2.3600,0.43513,4.982772,0.093812,2.368859,0.147705


### Calculating the Robust Value Score

In [19]:
for row in rv_stocks.index:
    percentiles = []
    for metric in metrics.values():
        percentiles.append(rv_stocks.loc[row, metric])
    rv_stocks.loc[row, "RV Score"] = mean(percentiles)
    
rv_stocks

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile
0,A,131.67,N/A,0.796208,31.96,0.730539,7.75,0.777445,6.0800,0.826347,23.205815,0.822355,12.040917,0.824351
1,AAL,14.90,N/A,0.226747,-5.20,0.031936,-1.19,0.061876,0.2451,0.011976,108.710243,0.996008,0.846311,0.031936
2,AAP,197.68,N/A,0.374451,21.72,0.516966,4.12,0.58483,1.1100,0.179641,12.651153,0.42515,2.610145,0.165669
3,AAPL,169.50,N/A,0.831936,28.44,0.648703,46.60,0.978044,6.9500,0.850299,20.905739,0.762475,15.830150,0.92016
4,ABBV,144.78,N/A,0.609182,20.02,0.46507,17.29,0.932136,4.3600,0.714571,11.398753,0.357285,7.791025,0.576846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.36,N/A,0.560878,22.27,0.522954,-4.16,0.057884,5.1200,0.782435,19.914102,0.740519,9.217570,0.700599
497,ZBH,117.87,N/A,0.594012,107.98,0.978044,1.96,0.289421,3.3400,0.568862,18.322700,0.704591,5.920571,0.429142
498,ZBRA,335.45,N/A,0.65509,34.66,0.760479,7.15,0.751497,3.0600,0.53992,17.828416,0.688623,7.323461,0.53493
499,ZION,57.29,N/A,0.19501,6.38,0.093812,1.58,0.204591,2.3600,0.43513,4.982772,0.093812,2.368859,0.147705


### Selecting the 50 Best Value Stocks¶

The lower the RV Score, the cheaper the stock so the better it is.

In [20]:
rv_stocks.sort_values(by="RV Score", inplace=True)
rv_stocks = rv_stocks[:50]
rv_stocks.reset_index(drop=True, inplace=True)
rv_stocks

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile
0,KSS,30.08,N/A,0.045908,4.65,0.051896,0.9100,0.091816,0.2122,0.007984,3.445444,0.033932,1.008209,0.043912
1,AIG,55.49,N/A,0.058283,4.87,0.057884,0.7757,0.077844,0.8120,0.111776,2.837772,0.023952,0.768230,0.01996
2,UNM,37.29,N/A,0.064671,7.02,0.111776,0.7659,0.075848,0.6284,0.07984,3.669094,0.045908,0.576181,0.00998
3,PVH,65.89,N/A,0.065868,4.58,0.0499,0.8488,0.083832,0.5053,0.053892,4.695975,0.085828,1.100592,0.055888
4,GPS,9.79,N/A,0.070459,-53.19,0.007984,1.5800,0.204591,0.2445,0.00998,5.386959,0.105788,0.781093,0.023952
5,GM,38.34,N/A,0.070659,7.00,0.10978,0.8615,0.085828,0.4250,0.03992,2.272716,0.017964,1.777883,0.0998
6,SYF,34.90,N/A,0.085828,5.16,0.06487,1.3500,0.148703,0.9614,0.143713,3.168267,0.025948,1.010638,0.045908
7,COF,110.40,N/A,0.087026,4.83,0.055888,0.7869,0.07984,1.2200,0.199601,3.342540,0.02994,1.290407,0.06986
8,LNC,47.10,N/A,0.091018,9.85,0.163673,0.8821,0.087824,0.4042,0.037924,6.880223,0.161677,0.404725,0.003992
9,NRG,39.09,N/A,0.097405,2.80,0.045908,1.7700,0.257485,0.3176,0.025948,2.748772,0.021956,2.230794,0.135729


### Calculating the Number of Shares to Buy

In [21]:
calculate_shares(rv_stocks)


Given your portfolio value at $123456789.0, you're position size is 2469135.78



c:\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [22]:
rv_stocks

,Ticker,Stock Price,No. of Shares to Buy,RV Score,P/E Ratio,P/E Percentile,P/B Ratio,P/B Percentile,P/S Ratio,P/S Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GS,EV/GS Percentile
0,KSS,30.08,82085.0,0.045908,4.65,0.051896,0.9100,0.091816,0.2122,0.007984,3.445444,0.033932,1.008209,0.043912
1,AIG,55.49,44496.0,0.058283,4.87,0.057884,0.7757,0.077844,0.8120,0.111776,2.837772,0.023952,0.768230,0.01996
2,UNM,37.29,66214.0,0.064671,7.02,0.111776,0.7659,0.075848,0.6284,0.07984,3.669094,0.045908,0.576181,0.00998
3,PVH,65.89,37473.0,0.065868,4.58,0.0499,0.8488,0.083832,0.5053,0.053892,4.695975,0.085828,1.100592,0.055888
4,GPS,9.79,252209.0,0.070459,-53.19,0.007984,1.5800,0.204591,0.2445,0.00998,5.386959,0.105788,0.781093,0.023952
5,GM,38.34,64401.0,0.070659,7.00,0.10978,0.8615,0.085828,0.4250,0.03992,2.272716,0.017964,1.777883,0.0998
6,SYF,34.90,70748.0,0.085828,5.16,0.06487,1.3500,0.148703,0.9614,0.143713,3.168267,0.025948,1.010638,0.045908
7,COF,110.40,22365.0,0.087026,4.83,0.055888,0.7869,0.07984,1.2200,0.199601,3.342540,0.02994,1.290407,0.06986
8,LNC,47.10,52423.0,0.091018,9.85,0.163673,0.8821,0.087824,0.4042,0.037924,6.880223,0.161677,0.404725,0.003992
9,NRG,39.09,63165.0,0.097405,2.80,0.045908,1.7700,0.257485,0.3176,0.025948,2.748772,0.021956,2.230794,0.135729


## Formatting Our Excel Output

In [23]:
writer = pd.ExcelWriter("value_strategy.xlsx", engine="xlsxwriter")
rv_stocks.to_excel(writer, "Value Strategy", index=False)

### Creating the Formats

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
    {
        "num_format": "0.0",
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats

Sample Code:

```python
writer.sheets['Recommended Trades'].set_column('B:B', # apply the format to column B
                     18, # column width of 18 pixels
                     string_template # applies the previously-built format 'string_template' to the column
                     )
```

In [25]:
column_formats = {
    "A": ["Ticker", string_template],
    "B": ["Stock Price", dollar_template],
    "C": ["No. of Shares to Buy", integer_template],
    "D": ["RV Score", percent_template],
    "E": ["P/E Ratio", float_template],
    "F": ["P/E Percentile", percent_template],
    "G": ["P/B Ratio", float_template],
    "H": ["P/B Percentile", percent_template],
    "I": ["P/S Ratio", float_template],
    "J": ["P/S Percentile", percent_template],
    "K": ["EV/EBITDA", float_template],
    "L": ["EV/EBITDA Percentile", percent_template],
    "M": ["EV/GS", float_template],
    "N": ["EV/GS Percentile", percent_template],
}

for col in column_formats:
    writer.sheets['Value Strategy'].set_column(f"{col}:{col}", 25, column_formats[col][1])
    writer.sheets['Value Strategy'].write(f'{col}1', column_formats[col][0], column_formats[col][1])

In [26]:
writer.save()